# Install packages

In [13]:
!pip install meeko rdkit

In [14]:
!pip install gemmi

In [15]:
!pip install meeko openbabel-wheel

In [16]:
!pip install vina

In [17]:
!pip install biopandas

In [ ]:
mount_drive = input("Would you like to access your google drive files? (y/n): ").strip().lower()
if mount_drive == "y" or mount_drive == "yes":
  print("Mounting drive")
  from google.colab import drive
  drive.mount('/content/drive')
else:
  print("Not mounting drive")

# Step 1: Generate or upload ligand .pdbqt file

In [2]:
from rdkit import Chem
from rdkit.Chem import AllChem
import meeko

ligand_file_source = input("Do you have a .pdbqt file for your ligand? (y/n): ").strip().lower()
if ligand_file_source == "y" or ligand_file_source =="yes":
  ligand_file = input("Enter the path to your ligand .pdbqt file: or type 'u' to upload ")
  if ligand_file == "u" or ligand_file == "U":
    from google.colab import files
    ligand_file = files.upload()
    print("Ligand file uploaded")
  else:
    print("File path saved.")
else:

  # Prompt user for ligand SMILES and filename
  ligand_smiles = input("Please enter the SMILES string for your ligand: ")
  output_filename_base = input("Please enter a filename (without extension) for the output PDBQT (e.g., my_ligand): ")
  output_filename = f"{output_filename_base}.pdbqt" # Automatically add .pdbqt extension

  # Create RDKit molecule from SMILES
  mol = Chem.MolFromSmiles(ligand_smiles)
  if mol is None:
      raise ValueError("Invalid SMILES string entered. Please check and try again.")

  mol = Chem.AddHs(mol)
  AllChem.EmbedMolecule(mol, AllChem.ETKDGv3())
  AllChem.MMFFOptimizeMolecule(mol)

  # Convert to PDBQT using Meeko
  preparator = meeko.MoleculePreparation()
  mol_setup = preparator.prepare(mol)[0]
  pdbqt_result_tuple = meeko.PDBQTWriterLegacy.write_string(mol_setup)
  pdbqt_string = pdbqt_result_tuple[0] # Extract the string from the tuple

  # Save as PDBQT
  with open(output_filename, "w") as f:
      f.write(pdbqt_string)
  ligand_file = output_filename
  print(f"Ligand successfully generated as {output_filename}")


/usr/local/lib/python3.12/dist-packages/jupyter_client/session.py:203: DeprecationWarning: datetime.datetime.utcnow() is deprecated and scheduled for removal in a future version. Use timezone-aware objects to represent datetimes in UTC: datetime.datetime.now(datetime.UTC).
  return datetime.utcnow().replace(tzinfo=utc)


Do you have a .pdbqt file for your ligand? (y/n): n
Please enter the SMILES string for your ligand: OC1=C(/C=N/CCCCCCC(=O)[O-])C(=CN1C)COP(=O)([O-])[O-]
Please enter a filename (without extension) for the output PDBQT (e.g., my_ligand): ligg
Ligand successfully generated as ligg.pdbqt


# Step 2: Prepare protein receptor
Alternatively, manually prep your receptor in PyMOL

In [10]:
# Interactive PDB Cleaning + Receptor Builder Cell

import requests, os, subprocess
from math import sqrt


print("This accept a PDB ID (recommended) or a .pdb file as input")

protein_file_source = input("Do you have a .pdb file for your protein? (y/n): ").strip().lower()

# Initialize output_name to ensure it's always defined
output_name = ""

if protein_file_source == "y" or protein_file_source == "yes":
  user_input_pdb_path = input("Enter the path to your protein .pdb file: or type 'u' to upload ")
  if user_input_pdb_path == "u" or user_input_pdb_path == "U":
    from google.colab import files
    uploaded_files = files.upload()
    # Assume single file upload, get the first filename
    pdb_filename = list(uploaded_files.keys())[0]
    pdb_file = f"/content/{pdb_filename}" # Full path to the uploaded file in Colab
    output_name = pdb_filename.replace(".pdb", "_clean.pdb") # Generate a cleaned output filename
    print(f"Uploaded file path for processing: {pdb_file}")
    print(f"Cleaned output will be saved as: {output_name}")
  else:
    print("File path saved.")
    pdb_file = user_input_pdb_path # The full path provided by the user
    # Extract the base filename and suggest a cleaned output name
    base_filename = os.path.basename(user_input_pdb_path)
    output_name = base_filename.replace(".pdb", "_clean.pdb") # Generate a cleaned output filename
    print(f"Input PDB file path: {pdb_file}")
    print(f"Cleaned output will be saved as: {output_name}")
else:
# Get PDB ID
  pdb_id = input("Enter PDB ID: ").lower()
  output_name_input = input("Output filename for cleaned receptor (default: receptor_clean.pdb): ").strip()

  if output_name_input == "":
      output_name = "receptor_clean.pdb"
  else:
      output_name = output_name_input
      # Ensure it ends with .pdb if not already
      if not output_name.lower().endswith(".pdb"): # Fix for extension if user provides name without it
          output_name += ".pdb"

# Download PDB
  url = f"https://files.rcsb.org/download/{pdb_id}.pdb"
  pdb_file = f"{pdb_id}.pdb" # This is the local filename for the downloaded PDB

  r = requests.get(url)
  if r.status_code != 200:
      raise ValueError(" Invalid PDB ID or download failed.")

  with open(pdb_file, "w") as f:
      f.write(r.text)

  print(f" Downloaded structure: {pdb_file}")

# Detect Ligands Automatically

ligands = set()

with open(pdb_file) as f:
    for line in f:
        if line.startswith("HETATM"):
            resn = line[17:20].strip()
            if resn != "HOH":
                ligands.add(resn)

print("\nDetected ligands / hetero residues:")
print("   ", ligands if ligands else "None found")


keep_metals = input("Keep metal ions if present? (y/n, default=y): ").strip().lower()
keep_metals = True if keep_metals in ["", "y", "yes"] else False

mode = input("Receptor mode: full or pocket? (full(f)/pocket(p), default=full): ").strip().lower()
if mode == "" or mode =='full' or mode =='f':
    mode = "full"
    pocket_def = input('Would you like to define a pocket center manually (m) or using a ligand (l) residue? (m/l, default=l): ').strip().lower()
    if pocket_def == "" or pocket_def == 'l':
      pocket_resn = input("Enter ligand residue name to define pocket center (e.g. PLP): ").strip().upper()
      pocket_radius = float(input("Pocket radius in Å (default=15): ") or 15)
      coords = []

      with open(pdb_file) as f:
          for line in f:
              if line.startswith("HETATM"):
                  resn = line[17:20].strip()
                  if resn == pocket_resn:
                      x = float(line[30:38])
                      y = float(line[38:46])
                      z = float(line[46:54])
                      coords.append((x, y, z))

      if not coords:
          raise ValueError(f" Could not find ligand {pocket_resn} to define pocket center.")

      # Average coordinate = center
      pocket_center = (
          sum(c[0] for c in coords) / len(coords),
          sum(c[1] for c in coords) / len(coords),
          sum(c[2] for c in coords) / len(coords),
      )

      print(f"\n Pocket center defined by ligand {pocket_resn}: {pocket_center}")
    else:
      manual_pocket_center = input("Define pocket center manually as [X, Y, Z] coordinates: ")


# Pocket parameters
if mode == "pocket" or mode =="p": # Changed to elif to avoid double prompting
    mode = "pocket"
    pocket_resn = input("Ligand residue name to define pocket center (e.g. PLP): ").strip().upper()
    pocket_radius = float(input("Pocket radius in Å (default=8): ") or 8)

# User Choose Ligands

remove_list = input(
    "\nEnter ligand residue names to REMOVE (comma-separated), or press Enter to remove ALL ligands: "
).strip()

if remove_list == "":
    ligands_to_remove = ligands
else:
    ligands_to_remove = {x.strip().upper() for x in remove_list.split(",")}

print("Ligands that will be removed:", ligands_to_remove)


# Metals Handling

metal_ions = {"ZN", "MG", "FE", "CA", "MN", "CU", "CO", "NI"}

# Pocket Center Detection (if needed)

if mode == "pocket":
    coords = []

    with open(pdb_file) as f:
        for line in f:
            if line.startswith("HETATM"):
                resn = line[17:20].strip()
                if resn == pocket_resn:
                    x = float(line[30:38])
                    y = float(line[38:46])
                    z = float(line[46:54])
                    coords.append((x, y, z))

    if not coords:
        raise ValueError(f" Could not find ligand {pocket_resn} to define pocket center.")

    # Average coordinate = center
    pocket_center = (
        sum(c[0] for c in coords) / len(coords),
        sum(c[1] for c in coords) / len(coords),
        sum(c[2] for c in coords) / len(coords),
    )

    print(f"\n Pocket center defined by ligand {pocket_resn}: {pocket_center}")
    print(f"Keeping residues within {pocket_radius} Å")


#  Write Output

def dist(a, b):
    return sqrt((a[0]-b[0])**2 + (a[1]-b[1])**2 + (a[2]-b[2])**2)

with open(pdb_file) as infile, open(output_name, "w") as outfile:
    for line in infile:

        # Protein atoms always allowed
        if line.startswith("ATOM"):
            if mode == "full":
                outfile.write(line)

            elif mode == "pocket":
                x = float(line[30:38])
                y = float(line[38:46])
                z = float(line[46:54])
                if dist((x, y, z), pocket_center) <= pocket_radius:
                    outfile.write(line)

        # HETATM handling
        elif line.startswith("HETATM"):
            resn = line[17:20].strip()

            # Remove unwanted ligands
            if resn in ligands_to_remove:
                continue

            # Keep metals if user wants
            if keep_metals and resn in metal_ions:
                outfile.write(line)

print(f"\nClean receptor saved as: {output_name}")

# Convert to PDBQT

convert = input("\nConvert cleaned receptor to PDBQT? (y/n, default=n): ").strip().lower()

if convert in ["y", "yes"]:
    pdbqt_name = output_name.replace(".pdb", ".pdbqt")

    subprocess.run([
        "obabel", output_name,
        "-O", pdbqt_name,
        "-xr",       # rigid receptor
        "-xp",       # preserve hydrogens if present
        "-h"         # add hydrogens
        ])

    print(f"Converted receptor saved as: {pdbqt_name}")

print("\nDone!")

This accept a PDB ID (recommended) or a .PDB file as input


KeyboardInterrupt: Interrupted by user

# Step3 Perform ligand docking and get ligand poses energies

In [7]:
from vina import Vina
import os
ligand_file_base = os.path.splitext(ligand_file)[0]

if pocket_def == "m":
  # Parse manual_pocket_center string into a list of floats for Vina's `center` argument
  center_coords_str = manual_pocket_center.strip('[]').split(',')
  center = [float(coord.strip()) for coord in center_coords_str]

  pocket_radius = float(input("Pocket radius in Å (default=15): ") or 15)


  # Define box_size as 2 * pocket_radius in each dimension, as typically used in Vina.
  box_dimension = 2 * pocket_radius
  box_size = [box_dimension, box_dimension, box_dimension]
else:
  # If pocket_def was 'l' (ligand-defined center), pocket_center is a tuple of floats.
  # Convert it to a list for Vina.
  center = list(pocket_center)
  # pocket_radius is already a float.
  box_dimension = 2 * pocket_radius
  box_size = [box_dimension, box_dimension, box_dimension]

def dock_and_score(receptor_pdbqt, ligand_pdbqt, center, box_size):
  print("Docking started ... This will take a few mins")
  v = Vina(sf_name='vina', cpu=4)
  v.set_receptor(receptor_pdbqt)
  v.set_ligand_from_file(ligand_pdbqt)
  v.compute_vina_maps(center=center, box_size=box_size)
  v.dock(exhaustiveness=32, n_poses=10)
  v.write_poses(f"{ligand_file_base}_poses.pdbqt", n_poses = 10)
  # Get energies for all poses
  energies = v.energies()
  # Each row: [total, inter, intra, torsional, inter-best]

  print(f"{'Pose':<6} {'Total':>8} {'Inter':>8} {'Intra':>8}")
  print("-" * 30)
  for i, e in enumerate(energies):
      print(f"{i+1:<6} {e[0]:>8.2f} {e[1]:>8.2f} {e[2]:>8.2f}")

  return energies

receptor_pdbqt = pdbqt_name
ligand_pdbqt = ligand_file

ligand_energies = dock_and_score(receptor_pdbqt, ligand_pdbqt, center, box_size)

# Compare best poses

print(f"best score: {ligand_energies[0][0]:.2f} kcal/mol")
print("Done!")

Docking started ... This will take a few mins
Pose      Total    Inter    Intra
------------------------------
1         -8.02   -13.41    -1.10
2         -7.27   -12.55    -0.71
3         -7.25   -12.37    -0.86
4         -7.19   -12.16    -0.97
5         -6.80   -11.83    -0.65
6         -6.72   -11.17    -1.17
7         -6.60   -11.84    -0.30
8         -6.56   -11.10    -0.97
9         -6.50   -11.08    -0.89
best score: -8.02 kcal/mol
Done!


# Step4: Get PDB file for ligand's poses

In [8]:
# Using Meeko to convert PDBQT back to PDB/SDF
from meeko import PDBQTMolecule, RDKitMolCreate
from rdkit import Chem




best_pose_only = input("Generate PDB for best pose only (1) or all poses (2)? ").strip().upper()

if best_pose_only == "1":
  best_pose_only = True
  # Convert best pose
  pdbqt_mol = PDBQTMolecule.from_file(f"{ligand_file_base}_poses.pdbqt")
  for i, pose in enumerate(pdbqt_mol):
      rd_mol = RDKitMolCreate.from_pdbqt_mol(pose)[0]
      Chem.MolToPDBFile(rd_mol, f"{ligand_file_base}_best_pose_{i}.pdb")
      if i == 0:
          break  # just the best pose
else:
  pdbqt_mol = PDBQTMolecule.from_file(f"{ligand_file_base}_poses.pdbqt")
  for i, pose in enumerate(pdbqt_mol):
      rd_mol = RDKitMolCreate.from_pdbqt_mol(pose)[0]
      Chem.MolToPDBFile(rd_mol, f"{ligand_file_base}_pose_{i}.pdb")


print("Ligand docking done, make sure your files are saved/downloaded")
print("rerun steps 1-4 again for a new protein-ligand pair")

Generate PDB for best pose only (1) or all poses (2)? 1
Ligand docking done, make sure your files are saved/downloaded
rerun steps 1-4 again for a new protein-ligand pair
